In [1]:
import json, os, requests

from key import API_KEY

API_KEY = API_KEY # Enter your API Key/"App ID" Here. Mine was 40 chars long.

# FILM_CAMS = '15230'
# ELEC_GUITARS = '33034' # Category code for electric guitars on eBay.
# WATCHES = '14324'
MENS_SUNGLASSES = '79720'

In [2]:
USA = 'EBAY-US' # USA Marketplace only. API seems to start returning stuff from other markets eventually anyhow
USED = '3000' # Just the code for item condition "used". This is the focus of this project.
AUCTIONWITHBIN = "AuctionWithBIN"
AUCTION = "Auction"

ITEM_FILTER_0 = f'itemFilter(0).name=Condition&itemFilter(0).value={USED}' # Only used guitars
ITEM_FILTER_1 = f'itemFilter(1).name=HideDuplicateItems&itemFilter(1).value=true' # No duplicate listings
ITEM_FILTER_2 = f'itemFilter(2).name=MinPrice&itemFilter(2).value=49' # Only items that sold for > this value
ITEM_FILTER_3 = f'itemFilter(3).name=MaxQuantity&itemFilter(3).value=1' # No lots or batch sales. One item at a time
ITEM_FILTER_4 = f'itemFilter(4).name=SoldItemsOnly&itemFilter(4).value=true' # Only looking at listings that sold.
ITEM_FILTER_5 = f'itemFilter(5).name=ListingType&itemFilter(5).value={AUCTION}' # Select sale type.
ITEM_FILTER_6 = f'itemFilter(6).name=MaxPrice&itemFilter(6).value=2000' # Only items that sold for < this value
ITEM_FILTER_7 = f'itemFilter(7).name=listingType&itemFilter(7).value={AUCTIONWITHBIN}' # Select sale type.

FIND_COMPLETED = 'findCompletedItems' # This is the eBay API endpoint service we'll be querying.

In [3]:
def find_completed_auction(PAGE):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_COMPLETED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
                 f'REST-PAYLOAD&'
                 f'GLOBAL-ID={USA}&' # seems to prioritize the value you enter but returns other stuff too
                 f'categoryId={MENS_SUNGLASSES}&' # Product category goes here
                 f'descriptionSearch=true&' # More verbose responses
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # ONLY SOLD
                 f'{ITEM_FILTER_5}&' # AUCTIONS ONLY
                 f'{ITEM_FILTER_6}&' # MAX PRICE
                 f'paginationInput.pageNumber={str(PAGE)}&' # value to be looped through when collecting lotsa data
                 f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool
    if r.json()['findCompletedItemsResponse'][0].get('searchResult'):
        return r.json()['findCompletedItemsResponse'][0]['searchResult'][0]['item']
    else:
        return None

In [4]:
find_completed_auction(1)

[{'itemId': ['264256128760'],
  'title': ['Vintage Sun Glasses Flip Up Green Lens Rectangular Accoutant Glasses Retro'],
  'globalId': ['EBAY-US'],
  'primaryCategory': [{'categoryId': ['79720'],
    'categoryName': ['Sunglasses']}],
  'galleryURL': ['http://thumbs1.ebaystatic.com/m/mJ-kEloE4bO0XhrrkxrL9hw/140.jpg'],
  'viewItemURL': ['http://www.ebay.com/itm/Vintage-Sun-Glasses-Flip-Up-Green-Lens-Rectangular-Accoutant-Glasses-Retro-/264256128760'],
  'paymentMethod': ['PayPal'],
  'autoPay': ['false'],
  'postalCode': ['67526'],
  'location': ['Ellinwood,KS,USA'],
  'country': ['US'],
  'shippingInfo': [{'shippingServiceCost': [{'@currencyId': 'USD',
      '__value__': '0.0'}],
    'shippingType': ['Free'],
    'shipToLocations': ['Worldwide'],
    'expeditedShipping': ['false'],
    'oneDayShippingAvailable': ['false'],
    'handlingTime': ['3']}],
  'sellingStatus': [{'currentPrice': [{'@currencyId': 'USD',
      '__value__': '50.0'}],
    'convertedCurrentPrice': [{'@currencyId': '

In [5]:
# For BUY IT NOW:
def find_completed_auction_BIN(PAGE):
    '''Make a request to the eBay API and return the JSON text of this page number'''
    r = requests.get(f'https://svcs.ebay.com/services/search/FindingService/v1?'
                 f'OPERATION-NAME={FIND_COMPLETED}&'
                 f'X-EBAY-SOA-SECURITY-APPNAME={API_KEY}&'
                 f'RESPONSE-DATA-FORMAT=JSON&' # This value can be altered if you're not into JSON responses
                 f'REST-PAYLOAD&'
                 f'GLOBAL-ID={USA}&' # seems to prioritize the value you enter but returns other stuff too
                 f'categoryId={MENS_SUNGLASSES}&' # Product category goes here
                 f'descriptionSearch=true&' # More verbose responses
                 f'{ITEM_FILTER_0}&' # USED
                 f'{ITEM_FILTER_1}&' # NO DUPES
                 f'{ITEM_FILTER_2}&' # MINPRICE
                 f'{ITEM_FILTER_3}&' # NO LOTS
                 f'{ITEM_FILTER_4}&' # ONLY SOLD
                 f'{ITEM_FILTER_7}&' # BUY IT NOW ONLY
                 f'paginationInput.pageNumber={str(PAGE)}&' # value to be looped through when collecting lotsa data
                 f'outputSelector=PictureURLLarge') # Why not grab the thumbnail URLs too. Could be cool
    if r.json()['findCompletedItemsResponse'][0].get('searchResult'):
        return r.json()['findCompletedItemsResponse'][0]['searchResult'][0]['item']
    else:
        return None

In [6]:
def get_specs(ITEM_ID):
    '''Return the specifics of a single eBay auction. String input.'''
    r2 = requests.get('http://open.api.ebay.com/shopping?'
                    f'callname=GetSingleItem&'
                    f'responseencoding=JSON&'
                    f'appid={API_KEY}&'
                    f'siteid=0&' # USA Store
                    f'version=967&' # What is this?
                    f'ItemID={ITEM_ID}&' # Assigned above
                    f'IncludeSelector=Details,ItemSpecifics,TextDescription')
    try:
        return r2.json()['Item']
    except KeyError:
        pass

In [7]:
# Seems like there's some variability between items when it comes to item specifics field.
# ### Persisting some Data for Analysis

# Just write first page of listing results to .json files:
def persist_page_to_json(PAGE):
    '''Saves a page of JSON responses to one json per sale / item'''
    for i in range(len(PAGE)):
        with open("data/listings/%s_listing.json" % (PAGE[i]['itemId'][0]), 'w') as f:  # writing JSON object
            json.dump(PAGE[i], f)

In [8]:
# Now write one page of details to a JSON:
def persist_spec_to_json(spec):
    '''Writes one page of Axe Specs to one json'''
    try:
        with open("data/specs/%s_specs.json" % (spec['ItemID']), 'w') as f:  # writing JSON object
            json.dump(spec, f)
    except TypeError:
        pass
    pass

In [9]:
# Okay, careful, this is where we start to hammer the eBay API a little bit.
def spam_the_api(start_page, stop_page, fetch_function):
    existing_files = [name.split('_')[0] for name in os.listdir('data/specs/') if not name.startswith('.')] # Ignore .DS_Store
    
    j = 0
    k = 0
    
    '''Spams the eBay API for pages of item DATA'''
    
    for i in range(start_page+1, stop_page+1):
        page = fetch_function(i)
        if page != None:
            persist_page_to_json(page)
            for item in page:
                k += 1
                if item['itemId'][0] not in existing_files:
                    j += 1
                    print(f'Get page {i} item {k}')
                    persist_spec_to_json(get_specs(item['itemId'][0]))
                else:
                    print(f'Skip page {i} item {k}')    
    print(f'\nChecked {k} items')
    print(f'\nGot {j} new items')

In [10]:
# Again, you only get 5k API calls per day.
spam_the_api(0,20, find_completed_auction)

Get page 1 item 1
Get page 1 item 2
Get page 1 item 3
Get page 1 item 4
Get page 1 item 5
Get page 1 item 6
Get page 1 item 7
Get page 1 item 8
Get page 1 item 9
Get page 1 item 10
Get page 1 item 11
Get page 1 item 12
Get page 1 item 13
Get page 1 item 14
Get page 1 item 15
Get page 1 item 16
Get page 1 item 17
Get page 1 item 18
Get page 1 item 19
Get page 1 item 20
Get page 1 item 21
Get page 1 item 22
Get page 1 item 23
Get page 1 item 24
Get page 1 item 25
Skip page 1 item 26
Skip page 1 item 27
Skip page 1 item 28
Skip page 1 item 29
Skip page 1 item 30
Skip page 1 item 31
Skip page 1 item 32
Skip page 1 item 33
Skip page 1 item 34
Skip page 1 item 35
Skip page 1 item 36
Skip page 1 item 37
Skip page 1 item 38
Skip page 1 item 39
Skip page 1 item 40
Skip page 1 item 41
Skip page 1 item 42
Skip page 1 item 43
Skip page 1 item 44
Skip page 1 item 45
Skip page 1 item 46
Skip page 1 item 47
Skip page 1 item 48
Skip page 1 item 49
Skip page 1 item 50
Skip page 1 item 51
Skip page 1 it

Get page 5 item 424
Get page 5 item 425
Get page 5 item 426
Get page 5 item 427
Get page 5 item 428
Get page 5 item 429
Get page 5 item 430
Get page 5 item 431
Get page 5 item 432
Get page 5 item 433
Get page 5 item 434
Get page 5 item 435
Get page 5 item 436
Get page 5 item 437
Get page 5 item 438
Get page 5 item 439
Get page 5 item 440
Get page 5 item 441
Get page 5 item 442
Get page 5 item 443
Get page 5 item 444
Get page 5 item 445
Get page 5 item 446
Get page 5 item 447
Get page 5 item 448
Get page 5 item 449
Get page 5 item 450
Get page 5 item 451
Get page 5 item 452
Get page 5 item 453
Get page 5 item 454
Get page 5 item 455
Get page 5 item 456
Get page 5 item 457
Get page 5 item 458
Get page 5 item 459
Get page 5 item 460
Get page 5 item 461
Get page 5 item 462
Get page 5 item 463
Get page 5 item 464
Get page 5 item 465
Get page 5 item 466
Get page 5 item 467
Get page 5 item 468
Get page 5 item 469
Get page 5 item 470
Get page 5 item 471
Get page 5 item 472
Get page 5 item 473


Get page 9 item 834
Get page 9 item 835
Get page 9 item 836
Get page 9 item 837
Get page 9 item 838
Get page 9 item 839
Get page 9 item 840
Get page 9 item 841
Get page 9 item 842
Get page 9 item 843
Get page 9 item 844
Get page 9 item 845
Get page 9 item 846
Get page 9 item 847
Get page 9 item 848
Get page 9 item 849
Get page 9 item 850
Get page 9 item 851
Get page 9 item 852
Get page 9 item 853
Get page 9 item 854
Get page 9 item 855
Get page 9 item 856
Get page 9 item 857
Get page 9 item 858
Get page 9 item 859
Get page 9 item 860
Get page 9 item 861
Get page 9 item 862
Get page 9 item 863
Get page 9 item 864
Get page 9 item 865
Get page 9 item 866
Get page 9 item 867
Get page 9 item 868
Get page 9 item 869
Get page 9 item 870
Get page 9 item 871
Get page 9 item 872
Get page 9 item 873
Get page 9 item 874
Get page 9 item 875
Get page 9 item 876
Get page 9 item 877
Get page 9 item 878
Get page 9 item 879
Get page 9 item 880
Get page 9 item 881
Get page 9 item 882
Get page 9 item 883


Get page 13 item 1217
Get page 13 item 1218
Get page 13 item 1219
Get page 13 item 1220
Get page 13 item 1221
Get page 13 item 1222
Get page 13 item 1223
Get page 13 item 1224
Get page 13 item 1225
Get page 13 item 1226
Get page 13 item 1227
Get page 13 item 1228
Get page 13 item 1229
Get page 13 item 1230
Get page 13 item 1231
Get page 13 item 1232
Get page 13 item 1233
Get page 13 item 1234
Get page 13 item 1235
Get page 13 item 1236
Get page 13 item 1237
Get page 13 item 1238
Get page 13 item 1239
Get page 13 item 1240
Get page 13 item 1241
Get page 13 item 1242
Get page 13 item 1243
Get page 13 item 1244
Get page 13 item 1245
Get page 13 item 1246
Get page 13 item 1247
Get page 13 item 1248
Get page 13 item 1249
Get page 13 item 1250
Get page 13 item 1251
Get page 13 item 1252
Get page 13 item 1253
Get page 13 item 1254
Get page 13 item 1255
Get page 13 item 1256
Get page 13 item 1257
Get page 13 item 1258
Get page 13 item 1259
Get page 13 item 1260
Get page 13 item 1261
Get page 1

Get page 16 item 1590
Get page 16 item 1591
Get page 16 item 1592
Get page 16 item 1593
Get page 16 item 1594
Get page 16 item 1595
Get page 16 item 1596
Get page 16 item 1597
Get page 16 item 1598
Get page 16 item 1599
Get page 16 item 1600
Get page 17 item 1601
Get page 17 item 1602
Get page 17 item 1603
Get page 17 item 1604
Get page 17 item 1605
Get page 17 item 1606
Get page 17 item 1607
Get page 17 item 1608
Get page 17 item 1609
Get page 17 item 1610
Get page 17 item 1611
Get page 17 item 1612
Get page 17 item 1613
Get page 17 item 1614
Get page 17 item 1615
Get page 17 item 1616
Get page 17 item 1617
Get page 17 item 1618
Get page 17 item 1619
Get page 17 item 1620
Get page 17 item 1621
Get page 17 item 1622
Get page 17 item 1623
Get page 17 item 1624
Get page 17 item 1625
Get page 17 item 1626
Get page 17 item 1627
Get page 17 item 1628
Get page 17 item 1629
Get page 17 item 1630
Get page 17 item 1631
Get page 17 item 1632
Get page 17 item 1633
Get page 17 item 1634
Get page 1

Get page 20 item 1964
Get page 20 item 1965
Get page 20 item 1966
Get page 20 item 1967
Get page 20 item 1968
Get page 20 item 1969
Get page 20 item 1970
Get page 20 item 1971
Get page 20 item 1972
Get page 20 item 1973
Get page 20 item 1974
Get page 20 item 1975
Get page 20 item 1976
Get page 20 item 1977
Get page 20 item 1978
Get page 20 item 1979
Get page 20 item 1980
Get page 20 item 1981
Get page 20 item 1982
Get page 20 item 1983
Get page 20 item 1984
Get page 20 item 1985
Get page 20 item 1986
Get page 20 item 1987
Get page 20 item 1988
Get page 20 item 1989
Get page 20 item 1990
Get page 20 item 1991
Get page 20 item 1992
Get page 20 item 1993
Get page 20 item 1994
Get page 20 item 1995
Get page 20 item 1996
Get page 20 item 1997
Get page 20 item 1998
Get page 20 item 1999
Get page 20 item 2000

Checked 2000 items

Got 1732 new items


In [11]:
spam_the_api(0, 20, find_completed_auction_BIN)

Skip page 1 item 1
Skip page 1 item 2
Get page 1 item 3
Skip page 1 item 4
Get page 1 item 5
Skip page 1 item 6
Get page 1 item 7
Get page 1 item 8
Get page 1 item 9
Skip page 1 item 10
Skip page 1 item 11
Skip page 1 item 12
Skip page 1 item 13
Get page 1 item 14
Get page 1 item 15
Get page 1 item 16
Get page 1 item 17
Get page 1 item 18
Get page 1 item 19
Skip page 1 item 20
Get page 1 item 21
Get page 1 item 22
Skip page 1 item 23
Skip page 1 item 24
Get page 1 item 25
Skip page 1 item 26
Skip page 1 item 27
Get page 1 item 28
Skip page 1 item 29
Skip page 1 item 30
Get page 1 item 31
Skip page 1 item 32
Skip page 1 item 33
Get page 1 item 34
Get page 1 item 35
Get page 1 item 36
Get page 1 item 37
Skip page 1 item 38
Get page 1 item 39
Get page 1 item 40
Get page 1 item 41
Skip page 1 item 42
Skip page 1 item 43
Get page 1 item 44
Get page 1 item 45
Get page 1 item 46
Get page 1 item 47
Skip page 1 item 48
Get page 1 item 49
Get page 1 item 50
Get page 1 item 51
Get page 1 item 52


Skip page 5 item 411
Get page 5 item 412
Get page 5 item 413
Skip page 5 item 414
Get page 5 item 415
Get page 5 item 416
Skip page 5 item 417
Get page 5 item 418
Get page 5 item 419
Get page 5 item 420
Get page 5 item 421
Skip page 5 item 422
Get page 5 item 423
Get page 5 item 424
Get page 5 item 425
Get page 5 item 426
Get page 5 item 427
Get page 5 item 428
Skip page 5 item 429
Get page 5 item 430
Skip page 5 item 431
Skip page 5 item 432
Get page 5 item 433
Get page 5 item 434
Get page 5 item 435
Get page 5 item 436
Get page 5 item 437
Get page 5 item 438
Skip page 5 item 439
Skip page 5 item 440
Skip page 5 item 441
Skip page 5 item 442
Get page 5 item 443
Get page 5 item 444
Get page 5 item 445
Skip page 5 item 446
Get page 5 item 447
Get page 5 item 448
Skip page 5 item 449
Get page 5 item 450
Get page 5 item 451
Skip page 5 item 452
Get page 5 item 453
Get page 5 item 454
Skip page 5 item 455
Get page 5 item 456
Skip page 5 item 457
Get page 5 item 458
Get page 5 item 459
Get 

KeyboardInterrupt: 